In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
# [Example] Read datasets
student_data = pd.read_csv("/data/training/studentlist.csv")

FileNotFoundError: [Errno 2] File b'/data/training/studentlist.csv' does not exist: b'/data/training/studentlist.csv'

In [ ]:
#Fetching and refining quiz1 dataset
quiz1_data = pd.read_csv("/data/training/quiz/quiz1.csv")
#print(quiz1_data.head(5))
quiz1_data_notNull = quiz1_data[pd.notnull(quiz1_data['State'])]
quiz1_data_trimmed = quiz1_data_notNull.drop(['ID number', 'Institution', 'Department',
       'Email address', 'Started on', 'Completed', 'Time taken', 'Q. 1 /1.00', 'Q. 2 /1.00', 'Q. 3 /1.00', 'Q. 4 /1.00',
       'Q. 5 /1.00', 'Q. 6 /1.00', 'Q. 7 /1.00', 'Q. 8 /1.00', 'Q. 9 /1.00',
       'Q. 10 /1.00'], axis=1)
#Replace non-numeric characters like '-' with 0 in 'Grade/10.00' column
quiz1_data_trimmed['Grade/10.00'] = (pd.to_numeric(quiz1_data_trimmed['Grade/10.00'], errors='coerce').fillna(0))
gradeList = quiz1_data_trimmed['Grade/10.00'].astype(str).astype(float)
quiz1_data_trimmed['percentage_quiz1'] = gradeList * 10

In [3]:
#Fetching and refining quiz2 dataset
quiz2_data = pd.read_csv("/data/training/quiz/quiz2.csv")
#print(quiz2_data.head(5))
quiz2_data_notNull = quiz2_data[pd.notnull(quiz2_data['State'])]
quiz2_data_trimmed = quiz2_data_notNull.drop(['ID number', 'Institution', 'Department',
       'Email address', 'Started on', 'Completed', 'Time taken', 'Q. 1 /1.00', 'Q. 2 /1.00', 'Q. 3 /1.00', 'Q. 4 /1.00',
       'Q. 5 /1.00', 'Q. 6 /1.00', 'Q. 7 /1.00', 'Q. 8 /1.00', 'Q. 9 /1.00',
       'Q. 10 /1.00'], axis=1)
#Replace non-numeric characters like '-' with 0 in 'Grade/10.00' column
quiz2_data_trimmed['Grade/10.00'] = (pd.to_numeric(quiz2_data_trimmed['Grade/10.00'], errors='coerce').fillna(0))
gradeList = quiz2_data_trimmed['Grade/10.00'].astype(str).astype(float)
quiz2_data_trimmed['percentage_quiz2'] = gradeList * 10

FileNotFoundError: [Errno 2] File b'/data/training/quiz/quiz2.csv' does not exist: b'/data/training/quiz/quiz2.csv'

In [4]:
testFile = open("/data/training/testcaseStudent.txt")
testcases = testFile.readlines()

FileNotFoundError: [Errno 2] No such file or directory: '/data/training/testcaseStudent.txt'

In [ ]:
line = testcases[0]
#print("line", line)
i = 0
for j in range(int(line.strip())):
    i += 1;
    batchNoIp = testcases[i].strip()
    #print("batchNoIp:", batchNoIp)
    i += 1;
    percentRangeIp = testcases[i].strip()
    #print("percentRangeIp:", percentRangeIp)
    ########################logic#################################
    batch_data = pd.read_csv("/data/training/batchwiselist/"+batchNoIp)
    batch_data['batch'] = 1
    #print(batch_data.head(5))
    batch_quiz1_data = pd.merge(batch_data, quiz1_data_trimmed, how='inner', left_on='studentName', right_on='Firstname').drop(['Unnamed: 0','Firstname', 'srNo'], axis=1)
    batch_quiz1_segregated = batch_quiz1_data.assign(percentRange=pd.cut(batch_quiz1_data.percentage_quiz1, bins=[0, 50, 60, 70, 80, np.inf], labels=['lessthan50', 'between50and60', 'between60and70', 'between70and80', 'greaterthan80'])).drop('percentage_quiz1', axis=1)
    batch_quiz1_segregated['noofpresent'] = batch_quiz1_segregated['batch'].count()
    batch_quiz1_segregated_trimmed = batch_quiz1_segregated.drop(['enrollmentNumber', 'batch', 'admissionNumber', 'studentName', 'Surname', 'State', 'Grade/10.00'], axis=1)
    batch_quiz1_pivot = pd.pivot_table(batch_quiz1_segregated_trimmed, index=['noofpresent'], columns=['percentRange'], fill_value=0.0, aggfunc=len)
    batch_quiz1_pivot = pd.DataFrame(batch_quiz1_pivot.to_records())
    
    batch_quiz2_data = pd.merge(batch_data, quiz2_data_trimmed, how='inner', left_on='studentName', right_on='Firstname').drop(['Unnamed: 0','Firstname', 'srNo'], axis=1)
    batch_quiz2_segregated = batch_quiz2_data.assign(percentRange=pd.cut(batch_quiz2_data.percentage_quiz2, bins=[0, 50, 60, 70, 80, np.inf], labels=['lessthan50', 'between50and60', 'between60and70', 'between70and80', 'greaterthan80'])).drop('percentage_quiz2', axis=1)
    batch_quiz2_segregated['noofpresent'] = batch_quiz2_segregated['batch'].count()
    batch_quiz2_segregated_trimmed = batch_quiz2_segregated.drop(['enrollmentNumber', 'batch', 'admissionNumber', 'studentName', 'Surname', 'State', 'Grade/10.00'], axis=1)
    batch_quiz2_pivot = pd.pivot_table(batch_quiz2_segregated_trimmed, index=['noofpresent'], columns=['percentRange'], fill_value=0.0, aggfunc=len)
    batch_quiz2_pivot = pd.DataFrame(batch_quiz2_pivot.to_records())
    
    batch_info = batch_quiz1_pivot.append(batch_quiz2_pivot, sort=True)
    batch_info = batch_info.fillna(0)
    
    output = batch_info[percentRangeIp].tolist()
    ##############################################################
    file = open("/code/output"+str(j+1)+".txt","w")
    for op in output:
        file.write(str(op))
    file.seek(0)
    file.close()
    #print("file closed", j+1)